# Collect data comment Youtube Rewind Indonesia 2015 dengan Google Client API

Import library

In [5]:
from dotenv import load_dotenv
import os
import googleapiclient.discovery
import pandas as pd
from pymongo import MongoClient

Mengakses value dari file env

In [6]:
# Memuat value dari file .env
load_dotenv()

dev = os.getenv('API_KEY_1')
mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

Inisialisasi videoID dan database name

In [7]:
video_id = "fNX6Gu2F7A8"
database_name = "youtube_rewind_indonesia"
collection_name = "collect_2015"

Proses collect data

In [8]:
# Konfigurasi nama layanan API dan versi yang akan digunakan
api_service_name = "youtube"
api_version = "v3"

DEVELOPER_KEY = dev

# Koneksi ke MongoDB
client = MongoClient(local_url)
db = client[database_name]
collection = db[collection_name]

# Membangun objek YouTube API dengan menggunakan kunci pengembang dan versi API yang ditentukan
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

# Membuat permintaan (request) untuk mendapatkan data komentar dari video tertentu
request = youtube.commentThreads().list(
    part="snippet",
    videoId=video_id,  # ID video yang akan diambil komentarnya
    maxResults=100  # Jumlah maksimum komentar yang akan diambil
)

# Inisialisasi list kosong untuk menyimpan komentar
comments = []

# Mengirimkan permintaan dan mendapatkan respons
response = request.execute()

counter = 1

# Mendapatkan komentar dari respons
for item in response['items']:
    comment = item['snippet']['topLevelComment']['snippet']
    public = item['snippet']['isPublic']
    # Menambahkan semua atribut komentar ke dalam list comments dalam bentuk dictionary
    comments.append({
        'authorDisplayName': comment['authorDisplayName'], # Nama penulis komentar
        'authorProfileImageUrl': comment['authorProfileImageUrl'], # URL gambar profil penulis
        'authorChannelUrl': comment['authorChannelUrl'], # URL channel penulis
        'channelId': comment['channelId'], # ID channel penulis
        'publishedAt': comment['publishedAt'], # Waktu publikasi komentar
        'updatedAt': comment['updatedAt'], # Waktu update komentar
        'likeCount': comment['likeCount'], # Jumlah suka komentar
        'textDisplay': comment['textDisplay'], # Isi komentar yang diformat dengan tautan dan emoji yang diproses
        'textOriginal': comment['textOriginal'], # Isi komentar
        'viewerRating': comment['viewerRating'], # Rating yang diberikan oleh pengguna
        'isPublic': public # Status publikasi komentar
    })

    # Menyisipkan data ke MongoDB
    collection.insert_one(comment)
    print(f"{counter} : {comment['textOriginal']}")
    counter+=1

# Melakukan iterasi untuk mengambil komentar dari halaman berikutnya jika ada
while (1 == 1):
    try:
        nextPageToken = response['nextPageToken']  # Mengambil token untuk halaman berikutnya
    except KeyError:
        break  # Keluar dari loop jika tidak ada halaman berikutnya
    
    nextPageToken = response['nextPageToken']  # Mengambil token untuk halaman berikutnya
    
    # Membuat permintaan baru dengan menggunakan token halaman berikutnya
    nextRequest = youtube.commentThreads().list(
        part="snippet", 
        videoId=video_id,  # ID video (opsional jika tetap sama dengan sebelumnya)
        maxResults=100,  # Jumlah maksimum komentar per halaman
        pageToken=nextPageToken  # Token untuk halaman berikutnya
    )
    # Mengirimkan permintaan baru dan mendapatkan respons
    response = nextRequest.execute()

    # Mendapatkan komentar dari respons dan menambahkannya ke dalam list comments
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        public = item['snippet']['isPublic']
        # Menambahkan semua atribut komentar ke dalam list comments dalam bentuk dictionary
        comments.append({
            'authorDisplayName': comment['authorDisplayName'], # Nama penulis komentar
            'authorProfileImageUrl': comment['authorProfileImageUrl'], # URL gambar profil penulis
            'authorChannelUrl': comment['authorChannelUrl'], # URL channel penulis
            'channelId': comment['channelId'], # ID channel penulis
            'publishedAt': comment['publishedAt'], # Waktu publikasi komentar
            'updatedAt': comment['updatedAt'], # Waktu update komentar
            'likeCount': comment['likeCount'], # Jumlah suka komentar
            'textDisplay': comment['textDisplay'], # Isi komentar yang diformat dengan tautan dan emoji yang diproses
            'textOriginal': comment['textOriginal'], # Isi komentar
            'viewerRating': comment['viewerRating'], # Rating yang diberikan oleh pengguna
            'isPublic': public # Status publikasi komentar
        })
        
        # Menyisipkan data ke MongoDB
        collection.insert_one(comment)
        print(f"{counter} : {comment['textOriginal']}")
        counter+=1

1 : Masih the best di hati oe
2 : 2024🎉
3 : 3:34 scene yg gw suka
4 : Diantara semua YouTube rewind, tahun ini yg paling favorit 😊😊
Bahkan tahun 2024 masih nonton, btw umur saya 22 tahun anak gen z yg rindu youtube rewind lama 😂
5 : ada yang balik kesini di 2024 ini rewind terbagus no debat❤😢
6 : Masih ada bryan disana
7 : ketika menonton rewind ini di masa saat ini(2024) serasa jadi time travelers, fakta memang dunia selalu berubah dan gak ketebak bakal menjadi apa, bersukur msh di kasih umur sampai saat ini😇🔥🔥🔥
8 : sumpah ini menurut gue rewind terbaik sih
9 : 2024 masih hadir
10 : era youtube terbaik, keluar dikit nih air mata kalo inget jaman itu, tiap hari nungguin salah satunya video arap
11 : Nge rewatch dan baru sadar ada rich brian 🗿
12 : 2024 dan masih mampir kesini, sehat-sehat ya kalian. tanpa sadar kalian telah menginspirasi kreator yang sekarang banyak di platform ini
13 : May 2024, and still watch this
14 : 2024 sni
15 : 3:22
16 : Dulu youtube rewind

Sekarang sirkel rew

In [9]:
# Membuat DataFrame dari list comments dengan kolom-kolom yang ditentukan
df = pd.DataFrame(comments)
# Menampilkan informasi DataFrame seperti jumlah baris dan tipe data kolom
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28414 entries, 0 to 28413
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   authorDisplayName      28414 non-null  object
 1   authorProfileImageUrl  28414 non-null  object
 2   authorChannelUrl       28414 non-null  object
 3   channelId              28414 non-null  object
 4   publishedAt            28414 non-null  object
 5   updatedAt              28414 non-null  object
 6   likeCount              28414 non-null  int64 
 7   textDisplay            28414 non-null  object
 8   textOriginal           28414 non-null  object
 9   viewerRating           28414 non-null  object
 10  isPublic               28414 non-null  bool  
dtypes: bool(1), int64(1), object(9)
memory usage: 2.2+ MB


In [10]:
df

,authorDisplayName,authorProfileImageUrl,authorChannelUrl,channelId,publishedAt,updatedAt,likeCount,textDisplay,textOriginal,viewerRating,isPublic
0,@KaylaAzzahra-fl9sf,https://yt3.ggpht.com/ytc/AIdro_n7417ek-18wzJI...,http://www.youtube.com/@KaylaAzzahra-fl9sf,UC-z1ubmS-iHMVr83EEwRS2g,2024-05-24T03:24:39Z,2024-05-24T03:24:39Z,0,Masih the best di hati oe,Masih the best di hati oe,none,True
1,@rezapratama391,https://yt3.ggpht.com/ytc/AIdro_kt7csXE65KUFFl...,http://www.youtube.com/@rezapratama391,UC-z1ubmS-iHMVr83EEwRS2g,2024-05-24T00:52:28Z,2024-05-24T00:52:28Z,0,2024🎉,2024🎉,none,True
2,@acesabo9179,https://yt3.ggpht.com/ytc/AIdro_lVQpw_BrtYzZYl...,http://www.youtube.com/@acesabo9179,UC-z1ubmS-iHMVr83EEwRS2g,2024-05-22T04:26:08Z,2024-05-22T04:26:08Z,0,"<a href=""https://www.youtube.com/watch?v=fNX6G...",3:34 scene yg gw suka,none,True
3,@WINHERO10,https://yt3.ggpht.com/2S9TD1tTD65kDm2AAWagbPeA...,http://www.youtube.com/@WINHERO10,UC-z1ubmS-iHMVr83EEwRS2g,2024-05-20T18:02:31Z,2024-05-20T18:06:29Z,2,"Diantara semua YouTube rewind, tahun ini yg pa...","Diantara semua YouTube rewind, tahun ini yg pa...",none,True
4,@nadyaamalia1486,https://yt3.ggpht.com/ytc/AIdro_koGD1zvhTLlTLo...,http://www.youtube.com/@nadyaamalia1486,UC-z1ubmS-iHMVr83EEwRS2g,2024-05-16T14:12:09Z,2024-05-16T14:12:09Z,0,ada yang balik kesini di 2024 ini rewind terba...,ada yang balik kesini di 2024 ini rewind terba...,none,True
...,...,...,...,...,...,...,...,...,...,...,...
28409,@GantplayBlogspot,https://yt3.ggpht.com/ytc/AIdro_nyQqIuJWFK6Ie5...,http://www.youtube.com/@GantplayBlogspot,UC-z1ubmS-iHMVr83EEwRS2g,2015-12-11T06:02:11Z,2015-12-11T06:02:11Z,0,Mantep :v,Mantep :v,none,True
28410,@ramandahafiz6293,https://yt3.ggpht.com/ytc/AIdro_nikMBBogL10baS...,http://www.youtube.com/@ramandahafiz6293,UC-z1ubmS-iHMVr83EEwRS2g,2015-12-11T06:02:07Z,2015-12-11T06:02:07Z,0,Boom !,Boom !,none,True
28411,@e4rtz84,https://yt3.ggpht.com/HWmDyGnG2Kzsde0_Z8ovBbWv...,http://www.youtube.com/@e4rtz84,UC-z1ubmS-iHMVr83EEwRS2g,2015-12-11T06:02:05Z,2015-12-11T06:02:05Z,1,anjing canggih waaahhhh,anjing canggih waaahhhh,none,True
28412,@pricillaoktaviani2376,https://yt3.ggpht.com/ytc/AIdro_lIX3-HU6zaCl0Z...,http://www.youtube.com/@pricillaoktaviani2376,UC-z1ubmS-iHMVr83EEwRS2g,2015-12-11T06:02:03Z,2015-12-11T06:02:03Z,0,firat,firat,none,True


In [11]:
df.sort_values(by='likeCount', ascending=False)[0:10]

,authorDisplayName,authorProfileImageUrl,authorChannelUrl,channelId,publishedAt,updatedAt,likeCount,textDisplay,textOriginal,viewerRating,isPublic
28002,@Tim2one,https://yt3.ggpht.com/ytc/AIdro_koWWb9JTasq6o_...,http://www.youtube.com/@Tim2one,UC-z1ubmS-iHMVr83EEwRS2g,2015-12-11T06:17:47Z,2015-12-11T06:17:47Z,15404,Ini semua kita persembahkan untuk kalian!,Ini semua kita persembahkan untuk kalian!,none,True
9680,@PUTRASIREGARMERAKYAT,https://yt3.ggpht.com/ytc/AIdro_mG2FOxqwZSIaxC...,http://www.youtube.com/@PUTRASIREGARMERAKYAT,UC-z1ubmS-iHMVr83EEwRS2g,2018-12-15T03:28:03Z,2018-12-15T03:28:03Z,7484,SIAPA YANG NONTON INI DI DESEMBER 2018 KARENA ...,SIAPA YANG NONTON INI DI DESEMBER 2018 KARENA ...,none,True
24841,@corbuzier,https://yt3.ggpht.com/EreG54tr2IsMffREgcPadhe6...,http://www.youtube.com/@corbuzier,UC-z1ubmS-iHMVr83EEwRS2g,2015-12-12T13:39:04Z,2015-12-12T13:39:04Z,7054,Very proud of all youngsters here.. Keep the g...,Very proud of all youngsters here.. Keep the g...,none,True
6501,@NessieJudge,https://yt3.ggpht.com/ytc/AIdro_mKhX_UMt80XsIz...,http://www.youtube.com/@NessieJudge,UC-z1ubmS-iHMVr83EEwRS2g,2020-05-16T18:28:51Z,2020-05-16T18:28:51Z,6578,♥️♥️♥️,♥️♥️♥️,none,True
4120,@iimpostor464,https://yt3.ggpht.com/ytc/AIdro_lKetRTmbWXYx2e...,http://www.youtube.com/@iimpostor464,UC-z1ubmS-iHMVr83EEwRS2g,2020-12-29T08:20:08Z,2020-12-29T08:20:08Z,6570,Siapa yg kesini karena udh liat yg YOUTUBE REW...,Siapa yg kesini karena udh liat yg YOUTUBE REW...,none,True
4869,@TeguhPrakoso27,https://yt3.ggpht.com/ytc/AIdro_m3I3qjg0pATXF6...,http://www.youtube.com/@TeguhPrakoso27,UC-z1ubmS-iHMVr83EEwRS2g,2020-12-11T12:41:09Z,2020-12-11T12:41:09Z,6122,good old time ~,good old time ~,none,True
4337,@galuhcuisine,https://yt3.ggpht.com/lZEqrz_0kCIAfIB_WKZ2Wm_-...,http://www.youtube.com/@galuhcuisine,UC-z1ubmS-iHMVr83EEwRS2g,2020-12-29T03:22:53Z,2020-12-29T03:22:53Z,5443,Nostalgia 😆,Nostalgia 😆,none,True
27535,@bayuekomoektito1,https://yt3.ggpht.com/ytc/AIdro_nFLECQuUs6Vlai...,http://www.youtube.com/@bayuekomoektito1,UC-z1ubmS-iHMVr83EEwRS2g,2015-12-11T06:52:58Z,2015-12-11T06:52:58Z,4787,Yeah! Hidup YOUTUBERS INDONESIA!,Yeah! Hidup YOUTUBERS INDONESIA!,none,True
27892,@skinnyindonesian24,https://yt3.ggpht.com/ytc/AIdro_lWjVa14wpop0lo...,http://www.youtube.com/@skinnyindonesian24,UC-z1ubmS-iHMVr83EEwRS2g,2015-12-11T06:24:14Z,2015-12-11T06:24:14Z,3884,Boom boom boom! Indonesian YouTubers gak kalah...,Boom boom boom! Indonesian YouTubers gak kalah...,none,True
5069,@19.kadekagussuardika76,https://yt3.ggpht.com/ytc/AIdro_mSDtUC3b-sim3A...,http://www.youtube.com/@19.kadekagussuardika76,UC-z1ubmS-iHMVr83EEwRS2g,2020-10-09T13:30:28Z,2020-10-09T13:30:28Z,3311,Brp orang yang Nostalgia merapat <br>👇,Brp orang yang Nostalgia merapat \n👇,none,True


Migrasi dari MongoDB Local ke MongoDB Atlas

In [12]:
# Koneksi ke MongoDB lokal
local_client = MongoClient(local_url)
local_db = local_client[database_name]
local_collection = local_db[collection_name]

# Koneksi ke MongoDB Atlas
atlas_client = MongoClient(mongodb_url)
atlas_db = atlas_client[database_name]
atlas_collection = atlas_db[collection_name]

# Ambil semua dokumen dari koleksi MongoDB lokal
documents = list(local_collection.find())

# Masukkan dokumen ke MongoDB Atlas
if documents:
    atlas_collection.insert_many(documents)
    print(f'{len(documents)} dokumen berhasil dipindahkan dari MongoDB lokal ke MongoDB Atlas.')
else:
    print('Tidak ada dokumen yang ditemukan di MongoDB lokal.')

# Tutup koneksi
local_client.close()
atlas_client.close()

ServerSelectionTimeoutError: SSL handshake failed: ac-bzbv7pi-shard-00-02.tux1wrd.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1006) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-bzbv7pi-shard-00-01.tux1wrd.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1006) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-bzbv7pi-shard-00-00.tux1wrd.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1006) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 66515e2330ea1caf68a16f5b, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-bzbv7pi-shard-00-00.tux1wrd.mongodb.net', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('SSL handshake failed: ac-bzbv7pi